In [1]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import seaborn as sns

import pycarrot as pc

%load_ext autoreload
%autoreload 2

pd.options.display.max_columns = 2000

sns.set_theme()

/home/george/Desktop/pycarrot/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# bc = load_breast_cancer()
# X = pd.DataFrame(bc.data, columns=bc.feature_names)
# y = pd.Series(bc.target, name="class")
# df = pd.concat([X, y], axis=1)
# df.head()

In [3]:
df = pd.read_csv(
    "./data/breast_cancer_cat/breast-cancer.data",
    names=[
        "class",
        "age",
        "menopause",
        "tumor-size",
        "inv-nodes",
        "node-caps",
        "deg-malig",
        "breast",
        "breast-quad",
        "irradiat",
    ],
)
df.head()

,class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [4]:
# pc.feat_analysis.get_distribution(df)

In [5]:
# sns.jointplot(data=df, x="deg-malig", y="breast", hue="class")

## Training

In [6]:
df.columns

Index(['class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps',
       'deg-malig', 'breast', 'breast-quad', 'irradiat'],
      dtype='object')

In [38]:
setup, X_sample, y_sample = pc.modelling.prepare_data(
    train_data=df,
    config_path="./config_bc_cat.yml",
)

In [39]:
X_sample

,deg-malig,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-79,menopause_ge40,menopause_lt40,menopause_premeno,tumor-size_0-4,tumor-size_10-14,tumor-size_15-19,tumor-size_20-24,tumor-size_25-29,tumor-size_30-34,tumor-size_35-39,tumor-size_40-44,tumor-size_45-49,tumor-size_5-9,tumor-size_50-54,inv-nodes_0-2,inv-nodes_12-14,inv-nodes_15-17,inv-nodes_24-26,inv-nodes_3-5,inv-nodes_6-8,inv-nodes_9-11,node-caps_?,node-caps_no,node-caps_yes,breast_left,breast_right,breast-quad_?,breast-quad_central,breast-quad_left_low,breast-quad_left_up,breast-quad_right_low,breast-quad_right_up,irradiat_no,irradiat_yes
0,3,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0
1,2,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0
2,2,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0
3,2,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0
4,2,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0


In [23]:
compare_df, algo_list, model_list = pc.modelling.compare_algorithms(
    setup=setup,
    include=[
        "lr",
        # "dt",
        # "rf",
        # "ridge",
        # "perceptron",
        # "passive-aggressive",
        # "extratree",
        # "extratrees",
        # "knn",
        "nb",
        # "linearsvc",
        # "rbfsvc",
    ],
    sort="f1",
    return_models=True,
)
compare_df

The algorithms ['lr'] work suboptimally without scaled features. Consider turning it on within the config.


,algorithm,accuracy,precision,recall,f1,roc_auc,Fit time (s)
0,nb,0.549,0.447,0.835,0.552,0.659,0.039
1,lr,0.678,0.527,0.329,0.338,0.653,0.186


In [12]:
le = setup.y_clf_encoder
preds = model_list[-1].predict(X_sample)
le.inverse_transform(preds)

array(['recurrence-events', 'no-recurrence-events',
       'no-recurrence-events', 'no-recurrence-events',
       'no-recurrence-events'], dtype=object)

In [13]:
print(*[(i, class_) for i, class_ in enumerate(le.classes_)])

(0, 'no-recurrence-events') (1, 'recurrence-events')


In [31]:
compare_df_tune, model_list = (
        pc.modelling.tune_hyperparams(
               setup=setup,
               include=["lr", "nb"],
               optimize="f1",
               n_trials=50,
               return_models=True,
        )
)
compare_df_tune

[I 2022-07-13 19:15:38,164] A new study created in memory with name: study_lr


The algorithms ['lr'] work suboptimally without scaled features. Consider turning it on within the config.


[I 2022-07-13 19:15:38,940] Trial 0 finished with value: 0.3937391304347827 and parameters: {'C': 2.071076459634078, 'l1_ratio': 0.609055362465034}. Best is trial 0 with value: 0.3937391304347827.
[I 2022-07-13 19:15:39,343] Trial 1 finished with value: 0.21429658685088696 and parameters: {'C': 0.20685031209237603, 'l1_ratio': 0.8238285006643981}. Best is trial 0 with value: 0.3937391304347827.
/home/george/Desktop/pycarrot/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2022-07-13 19:15:41,164] Trial 2 finished with value: 0.3702486402486403 and parameters: {'C': 89.13342908016536, 'l1_ratio': 0.6283096993478907}. Best is trial 0 with value: 0.3937391304347827.
[I 2022-07-13 19:15:41,760] Trial 3 finished with value: 0.3953201581027669 and parameters: {'C': 1.4262844645058486, 'l1_ratio': 0.1824226112859152}. Best is trial 3 with value: 0.3953201581027669.
[I 202

,algorithm,metric,hyperparams
0,nb,0.577910,"{'priors': None, 'var_smoothing': 7.3978247704..."
1,lr,0.401234,"{'C': 3.131676237250579, 'class_weight': None,..."


In [47]:
best_feature_list = pc.modelling.reduce_feature_space(
    setup=setup,
    algorithm= "nb",
    metric="f1",
    reference_metric=compare_df_tune.iloc[0]["metric"],
    acceptable_loss=0.5,
    hyperparams=compare_df_tune.iloc[0]["hyperparams"]
)

GaussianNB(var_smoothing=7.5566077239176915e-06)

In [35]:
compare_df_final, model_list = (
        pc.modelling.tune_hyperparams(
               setup=setup,
               include=["lr"],
               optimize="f1",
               n_trials=50,
               feature_list=best_feature_list,
               return_models=True,
        )
)
compare_df_final

[I 2022-07-13 19:34:17,478] A new study created in memory with name: study_lr


The algorithms ['lr'] work suboptimally without scaled features. Consider turning it on within the config.


[I 2022-07-13 19:34:17,735] Trial 0 finished with value: 0.0 and parameters: {'C': 4.681515509934075e-05, 'l1_ratio': 0.8297630704444257}. Best is trial 0 with value: 0.0.
[I 2022-07-13 19:34:18,098] Trial 1 finished with value: 0.12592592592592594 and parameters: {'C': 0.07709609773907886, 'l1_ratio': 0.2537021311573958}. Best is trial 1 with value: 0.12592592592592594.
[I 2022-07-13 19:34:18,491] Trial 2 finished with value: 0.4600204325502606 and parameters: {'C': 44.395960886014514, 'l1_ratio': 0.14261295761036574}. Best is trial 2 with value: 0.4600204325502606.
[I 2022-07-13 19:34:19,071] Trial 3 finished with value: 0.4643393694273371 and parameters: {'C': 16.601722579394856, 'l1_ratio': 0.2697119100306742}. Best is trial 3 with value: 0.4643393694273371.
[I 2022-07-13 19:34:19,365] Trial 4 finished with value: 0.0 and parameters: {'C': 0.01197983671221424, 'l1_ratio': 0.487488435842936}. Best is trial 3 with value: 0.4643393694273371.
[I 2022-07-13 19:34:19,688] Trial 5 finishe

,algorithm,metric,hyperparams
0,lr,0.469323,"{'C': 2.9756235554175228, 'class_weight': None..."
